In [1]:
import pandas as pd

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import re
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px

In [3]:
cr = pd.read_csv('./data/naver_crawler.csv')

In [4]:
pd = pd.read_csv('./data/product_df_total.csv')

In [5]:
cr.head()

,대분류,소분류,cate_url_middle
0,스킨케어,넥케어,https://search.shopping.naver.com/best100v2/de...
1,스킨케어,스킨/토너,https://search.shopping.naver.com/best100v2/de...
2,스킨케어,로션,https://search.shopping.naver.com/best100v2/de...
3,스킨케어,에센스,https://search.shopping.naver.com/best100v2/de...
4,스킨케어,크림,https://search.shopping.naver.com/best100v2/de...


In [6]:
pd.head()

,대분류,소분류,product_rank,product_title,product_price,product_review,product_url
0,스킨케어,넥케어,\n best\n ...,\n 벨라수 기린 넥프팅 크림 45g\n,"\n 최저6,250원\n","상품평 (2,925)",https://cr2.shopping.naver.com/adcr.nhn?x=uB0Q...
1,스킨케어,넥케어,\n best\n ...,\n 메디필 나이테 실 넥크림 100ml\n,"\n 최저8,980원\n","상품평 (2,663)",https://cr2.shopping.naver.com/adcr.nhn?x=p8Ck...
2,스킨케어,넥케어,\n best\n ...,\n 벨라코스메틱 넥크림 프레스티지 에이지 킬...\n,"\n 최저13,680원\n",상품평 (86),https://cr2.shopping.naver.com/adcr.nhn?x=xv%2...
3,스킨케어,넥케어,\n best\n ...,\n 이니스프리 자연발효 에너지 넥크림 80ml\n,"\n 최저11,140원\n",상품평 (430),https://cr2.shopping.naver.com/adcr.nhn?x=ieNq...
4,스킨케어,넥케어,\n best\n ...,\n 네이처카인드 인텐스 리프팅 넥크림 50g\n,"\n 최저14,320원\n","상품평 (1,019)",https://cr2.shopping.naver.com/adcr.nhn?x=alAQ...


In [7]:
cr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   대분류              120 non-null    object
 1   소분류              120 non-null    object
 2   cate_url_middle  120 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [8]:
pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9267 entries, 0 to 9266
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   대분류             9267 non-null   object
 1   소분류             9267 non-null   object
 2   product_rank    9267 non-null   object
 3   product_title   9267 non-null   object
 4   product_price   9267 non-null   object
 5   product_review  9267 non-null   object
 6   product_url     9267 non-null   object
dtypes: object(7)
memory usage: 506.9+ KB


In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import re
import pandas as pd
from datetime import datetime, timedelta

# https://search.shopping.naver.com/best100v2/detail.nhn?catId=50000190

category_df = pd.DataFrame({
    'category' : ['스킨케어', '베이스메이크업', '색조메이크업', '클렌징', 
                  '마스크/팩', '선케어', '남성화장품', '향수', 
                  '바디케어', '헤어케어', '헤어스타일링', '네일케어',
                 '뷰티소품'],
    'category_code' : ['50000190', '50000194', '50000195', '50000192', 
                       '50000193','50000191', '50000202', '50000200',
                      '50000197', '50000198', '50000199', '50000196',
                      '50000201']
})

total = pd.DataFrame()

today_dt = datetime.today()

for catid in range(len(category_df)):
    category_nm = category_df['category'][catid]
    category_code = category_df['category_code'][catid]

    url = 'https://search.shopping.naver.com/best100v2/detail.nhn?catId=' + category_code
    
    
    html = urlopen(url)
    bsObject = BeautifulSoup(html, "html.parser")

    tag = 'li'
    className = '_itemSection'
    content = bsObject.body.find_all(tag,{"class", className})


    # for i in range(3):
    for i in range(len(content)):
        tag = "p"
        className = "cont"
        title = content[i].find(tag,{"class", className})
        title_name = title.text.replace('\n',"")

        tag = "div"
        className = "price"
        price = content[i].find(tag,{"class", className})
        price_name = price.text.replace('\n','')
        
        price_num = numbers = re.sub(r'[^0-9]', '', price_name)
        
        df = pd.DataFrame({
            'date' : [today_dt],
            'category' : [category_nm],
            'title' : [title_name],
            'price' : [price_num],
        })
        total = pd.concat([total, df])

# 저장
total.to_csv("top_100.csv", index = False)

# # .to_csv 
# # 최초 생성 이후 mode는 append
# if not os.path.exists('top_100.csv'):
#     df.to_csv('top_100.csv', index=False, mode='w')
# else:
#     df.to_csv('top_100.csv', index=False, mode='a', header=False)


In [10]:
total

,date,category,title,price
0,2021-08-14 15:56:03.534969,스킨케어,가히 링클 바운스 멀티밤 스틱 9g,24600
0,2021-08-14 15:56:03.534969,스킨케어,설화수 자음 2종 세트,56920
0,2021-08-14 15:56:03.534969,스킨케어,AHC 유스래스팅 리얼 아이크림 포페이스 30...,2000
0,2021-08-14 15:56:03.534969,스킨케어,반코르 세라마이드 크림 60ml,13500
0,2021-08-14 15:56:03.534969,스킨케어,강블리 알로에 베라 모이스처 에멀전 150ml,2100
...,...,...,...,...
0,2021-08-14 15:56:03.534969,뷰티소품,그레이거 페이스필름 100매,6800
0,2021-08-14 15:56:03.534969,뷰티소품,에크멀 이지 브러쉬 에이에이 필 페이스 14,21240
0,2021-08-14 15:56:03.534969,뷰티소품,피카소 블랙 머쉬멜로우 메이크업 스펀지,8900
0,2021-08-14 15:56:03.534969,뷰티소품,피카소꼴레지오니 131 파운데이션 브러쉬,17060


In [11]:
category_df = pd.DataFrame({
    'category' : ['스킨케어', '베이스메이크업', '색조메이크업', '클렌징', 
                  '마스크/팩', '선케어', '남성화장품', '향수', 
                  '바디케어', '헤어케어', '헤어스타일링', '네일케어',
                 '뷰티소품'],
    'category_code' : ['50000190', '50000194', '50000195', '50000192', 
                       '50000193','50000191', '50000202', '50000200',
                      '50000197', '50000198', '50000199', '50000196',
                      '50000201']
})

total = pd.DataFrame()

today_dt = datetime.today()

In [12]:
category_df

,category,category_code
0,스킨케어,50000190
1,베이스메이크업,50000194
2,색조메이크업,50000195
3,클렌징,50000192
4,마스크/팩,50000193
5,선케어,50000191
6,남성화장품,50000202
7,향수,50000200
8,바디케어,50000197
9,헤어케어,50000198


In [13]:
catid = 0
category_nm = category_df['category'][catid]
category_code = category_df['category_code'][catid]

url = 'https://search.shopping.naver.com/best100v2/detail.nhn?catId=' + category_code

In [14]:
html = urlopen(url)
bsObject = BeautifulSoup(html, "html.parser")

In [15]:
tag = 'li'
className = '_itemSection'

content = bsObject.body.find_all(tag,{"class", className})

In [16]:
i = 0


In [17]:
tag = "p"
className = "cont"
title = content[i].find(tag,{"class", className})
title_name = title.text.replace('\n',"")

title_name

'가히 링클 바운스 멀티밤 스틱 9g'

In [18]:
tag = "div"
className = "price"
price = content[i].find(tag,{"class", className})
price_name = price.text.replace('\n','')
        
price_num = re.sub(r'[^0-9]', '', price_name)
price_num

'24600'

In [19]:
for i in range(len(content)):
        tag = "p"
        className = "cont"
        title = content[i].find(tag,{"class", className})
        title_name = title.text.replace('\n',"")

        tag = "div"
        className = "price"
        price = content[i].find(tag,{"class", className})
        price_name = price.text.replace('\n','')
        
        price_num = numbers = re.sub(r'[^0-9]', '', price_name)
        
        df = pd.DataFrame({
            'date' : [today_dt],
            'category' : [category_nm],
            'title' : [title_name],
            'price' : [price_num],
        })
        total = pd.concat([total, df])

In [20]:
total

,date,category,title,price
0,2021-08-14 15:57:10.723303,스킨케어,가히 링클 바운스 멀티밤 스틱 9g,24600
0,2021-08-14 15:57:10.723303,스킨케어,설화수 자음 2종 세트,56920
0,2021-08-14 15:57:10.723303,스킨케어,AHC 유스래스팅 리얼 아이크림 포페이스 30...,2000
0,2021-08-14 15:57:10.723303,스킨케어,반코르 세라마이드 크림 60ml,13500
0,2021-08-14 15:57:10.723303,스킨케어,강블리 알로에 베라 모이스처 에멀전 150ml,2100
...,...,...,...,...
0,2021-08-14 15:57:10.723303,스킨케어,비욘드 엔젤 아쿠아 수분 크림 150ml,6990
0,2021-08-14 15:57:10.723303,스킨케어,인셀덤 엑티브 크림 50ml,40640
0,2021-08-14 15:57:10.723303,스킨케어,싸이닉 더 심플 데일리 로션 260ml,7150
0,2021-08-14 15:57:10.723303,스킨케어,청미정 선인장 앰플 50ml,35000
